In [2]:
import numpy as np
from scipy.interpolate import interp1d

In [3]:
wavelength = 3.2e-02  # Довжина хвилі в метрах
a_r = 13e-02   # Розмір рупора по осі a в метрах
b_r = 13e-02   # Розмір рупора по осі b в метрах
theta = np.arange(0.01, np.pi / 2, 0.01)  # Кути theta для розрахунків

In [4]:
#Векторні обчислення
def calculate_FH(theta, wavelength, a_r):
    F1h = abs((1 + np.cos(theta)) / 2)
    FC = abs(np.cos((np.pi * a_r * np.sin(theta)) / wavelength) / 
             (1 - ((2 * a_r * np.sin(theta)) / wavelength)**2))
    FH = F1h * FC
    return F1h, FC, FH

def calculate_FE(theta, wavelength, b_r):
    F1e = abs((1 + np.cos(theta)) / 2)
    FC = abs(np.sin((np.pi * b_r * np.sin(theta)) / wavelength) / 
             ((np.pi * b_r * np.sin(theta)) / wavelength))
    FE = F1e * FC
    return F1e, FC, FE

In [5]:
#Площини H та E
F1h, FC_H, FH = calculate_FH(theta, wavelength, a_r)
F1e, FC_E, FE = calculate_FE(theta, wavelength, b_r)

In [6]:
#Інтерполюємо для точного визначення ширини головної пелюстки
interp_FH = interp1d(FH, theta, kind='linear', bounds_error=False, fill_value='extrapolate')
interp_FE = interp1d(FE, theta, kind='linear', bounds_error=False, fill_value='extrapolate')

In [7]:
#Ширина головної пелюстки
SGP1 = 2 * np.degrees(interp_FH(0.707))
SGP2 = 2 * np.degrees(interp_FE(0.707))

In [9]:
# Векторне знаходження максимумів та мінімумів
def find_peaks_and_troughs(F, theta):
    # Обчислюємо різниці між сусідніми значеннями
    d_F = np.diff(F)
    
    # Знаходимо індекси максимумів: де різниця змінюється з позитивної на негативну
    peaks_idx = np.where((d_F[:-1] > 0) & (d_F[1:] < 0))[0] + 1
    
    # Знаходимо індекси мінімумів: де різниця змінюється з негативної на позитивну
    troughs_idx = np.where((d_F[:-1] < 0) & (d_F[1:] > 0))[0] + 1
    
    # Перетворюємо індекси на кути θ та відповідні значення функції
    peaks_x = np.round(np.degrees(theta[peaks_idx]), 2)
    peaks_y = F[peaks_idx]
    troughs_x = np.round(np.degrees(theta[troughs_idx]), 2)
    troughs_y = np.zeros_like(troughs_x)
    
    return peaks_x, peaks_y, troughs_x, troughs_y

# Знаходимо максимуми і мінімуми для площин H та E
max_x_FH, max_y_FH, min_x_FH, min_y_FH = find_peaks_and_troughs(FH, theta)
max_x_FE, max_y_FE, min_x_FE, min_y_FE = find_peaks_and_troughs(FE, theta)

print(f"Значення нульових кутів в площині H: {', '.join(map(str, min_x_FH))} \u00b0")
print(f"Значення нульових кутів в площині E: {', '.join(map(str, min_x_FE))} \u00b0")

Значення нульових кутів в площині H: 21.77, 37.82, 59.59 °
Значення нульових кутів в площині E: 14.32, 29.22, 47.56, 80.21 °
